In [21]:
import spacy
import markovify
import import_ipynb
import dependency
from dependency import Dependency
import simplenlg as nlg
import frame
from frame import Frame
import random
import padawian_list
import constraint
from constraint import *

In [22]:
nlp = spacy.load('en_core_web_sm')
lexicon = nlg.Lexicon.getDefaultLexicon()
nlg_factory = nlg.NLGFactory(lexicon)
realiser = nlg.Realiser(lexicon)

In [23]:
class ChainMarkov(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.dep_)) for word in nlp(sentence)]
    
    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [24]:
class GenerateText:
    def __init__(self, corpus):
        self.padawian = random.choice(padawian_list.padawians) #si decide di parlare di una categoria
        self.frame = frame.Frame(self.padawian)
        self.text_model = self.read_text(corpus)
        self.text_model.compile(inplace=True)
        self.personaggio = random.choice(PERSONAGGIO)
    
    def read_text(self, corpus):
        if corpus == 'questions':
            with open('data/questions.txt') as f:
                text = f.read()
            
                #build model
                text_model = markovify.Text(text)
                #meglio con chainmarkov    
                text_model = ChainMarkov(text)
        
        return text_model
    
    
#NLG
    #Genero una frase di Presentazione introduttiva
    def introduzione(self):
        if self.personaggio == 'Obi Wan Kenobi':
            sent = nlg_factory.createClause(f"{random.choice(SALUTO)} Youngling,")
            subj = nlg_factory.createNounPhrase("I")
            verb = nlg_factory.createVerbPhrase("am")
            obj = nlg_factory.createNounPhrase(f"{self.personaggio}")

            sent_1 = nlg_factory.createClause(subj, verb, obj)

            coord = nlg_factory.createCoordinatedPhrase()
            coord.setConjunction(",")
            coord.addCoordinate(sent)
            coord.addCoordinate(sent_1)

            output = realiser.realiseSentence(coord)
        
        elif self.personaggio == 'Darth Fener':
            sent = nlg_factory.createClause(f"{random.choice(SALUTO_DARTH)}")
            verb = nlg_factory.createVerbPhrase("be")
            subj = nlg_factory.createNounPhrase("I")
            obj = nlg_factory.createNounPhrase(f"{self.personaggio}")

            sent_1 = nlg_factory.createClause(subj, verb, obj)

            coord = nlg_factory.createCoordinatedPhrase()
            coord.setConjunction(",")
            coord.addCoordinate(sent)
            coord.addCoordinate(sent_1)

            output = realiser.realiseSentence(coord)

        return output
    
    #Faccio partire l'intervista in maniera completamente random
    def start_questions(self):
        subj = nlg_factory.createNounPhrase("we")
        verb = nlg_factory.createVerbPhrase("talk")
        verb.setFeature(nlg.Feature.PERSON, nlg.Person.SECOND)
        verb.addPostModifier(f"about {self.padawian.get_name()}")
        sent = nlg_factory.createClause(subj, verb)

        sent_1 = nlg_factory.createClause("Well Youngling...")
        coord = nlg_factory.createCoordinatedPhrase()
        #coord.setConjunction(",")
        coord.addCoordinate(sent_1)
        coord.addCoordinate(sent)

        output = realiser.realiseSentence(coord)

        return output

    
#MARKOVIFY

    #generating some questions for padawians
    def generate_text(self):
        question = self.text_model.make_sentence(max_overlap=0.9, min_words=3, max_words=11) #genero la domanda

        #se la domanda non è generata, la rigenero
        if question is None:
            return self.generate_text()
        
        test = Dependency(question)
         
        #siccome le domande generate possono creare ridondanza, cerchiamo di filtrarle un po. Faccio tutte quelle domande formulate da solo una caratteristica
        while test.features_in_text() == 1:
            question = self.text_model.make_sentence(max_overlap=0.9, min_words=3, max_words=11)
            test = Dependency(question)
        
        #evito che si possano fare domande uguali
            if question[0] == question[1]:
                return self.generate_text()
        
        return question 


    def generate_good_answers(self):
        if self.personaggio == 'Obi Wan Kenobi':
            
            sent = nlg_factory.createClause(f"{random.choice(CLAUSE_OBI_POSITIVE)},")
            subj = nlg_factory.createNounPhrase("it")
            verb = nlg_factory.createVerbPhrase("be")
            obj = nlg_factory.createNounPhrase(f"the {random.choice(POSITIVE)} answer", f"of this {random.choice(FEATURES)}")

            sent_1 = nlg_factory.createClause(subj, verb, obj)

            coord = nlg_factory.createCoordinatedPhrase()
            coord.setConjunction(",")
            coord.addCoordinate(sent)
            coord.addCoordinate(sent_1)

            output = realiser.realiseSentence(coord)
        
        elif self.personaggio == 'Darth Fener':
            sent = nlg_factory.createClause(f"{random.choice(CLAUSE_DARTH_POSITIVE)},")
            subj = nlg_factory.createNounPhrase("it")
            verb = nlg_factory.createVerbPhrase("be")
            obj = nlg_factory.createNounPhrase(f"the {random.choice(POSITIVE)} answer", f"of this {random.choice(FEATURES)}")

            sent_1 = nlg_factory.createClause(subj, verb, obj)

            coord = nlg_factory.createCoordinatedPhrase()
            coord.setConjunction(",")
            coord.addCoordinate(sent)
            coord.addCoordinate(sent_1)

            output = realiser.realiseSentence(coord)

        return output
    
    def generate_negative_answers(self):
        if self.personaggio == 'Obi Wan Kenobi':
            
            sent = nlg_factory.createClause(f"{random.choice(CLAUSE_OBI_NEGATIVE)},")
            subj = nlg_factory.createNounPhrase("it")
            verb = nlg_factory.createVerbPhrase("be")
            obj = nlg_factory.createNounPhrase(f"the {random.choice(NEGATIVE)} answer", f"of this {random.choice(FEATURES)}")

            sent_1 = nlg_factory.createClause(subj, verb, obj)

            coord = nlg_factory.createCoordinatedPhrase()
            coord.setConjunction(",")
            coord.addCoordinate(sent)
            coord.addCoordinate(sent_1)

            output = realiser.realiseSentence(coord)
        
        elif self.personaggio == 'Darth Fener':
            sent = nlg_factory.createClause(f"{random.choice(CLAUSE_DARTH_NEGATIVE)},")
            subj = nlg_factory.createNounPhrase("it")
            verb = nlg_factory.createVerbPhrase("be")
            obj = nlg_factory.createNounPhrase(f"the {random.choice(NEGATIVE)} answer", f"of this {random.choice(FEATURES)}")

            sent_1 = nlg_factory.createClause(subj, verb, obj)

            coord = nlg_factory.createCoordinatedPhrase()
            coord.setConjunction(",")
            coord.addCoordinate(sent)
            coord.addCoordinate(sent_1)

            output = realiser.realiseSentence(coord)

        return output

"""
    #Possibile lista delle Risposte positive che si possono dare quando una risposta e' corretta
    def generate_good_answer(self):
            n = random.randint(0, 2)
            if n == 2:
                sent_1 = nlg_factory.createClause("Ok,")
                subj = nlg_factory.createNounPhrase("it")
                verb = nlg_factory.createVerbPhrase("be")
                obj = nlg_factory.createNounPhrase(f"the {random.choice(POSITIVE)} answer")

                sent = nlg_factory.createClause(subj, verb, obj)

                coord = nlg_factory.createCoordinatedPhrase()
                coord.setConjunction(",")
                coord.addCoordinate(sent_1)
                coord.addCoordinate(sent)

                output = realiser.realiseSentence(coord)
        
            elif n == 1:
                sent_1 = nlg_factory.createClause("Yes")
                subj = nlg_factory.createNounPhrase("it")
                verb = nlg_factory.createVerbPhrase("be")
                obj = nlg_factory.createNounPhrase(f"the {random.choice(POSITIVE)} answer of this", f"{random.choice(FEATURES)}")

                sent = nlg_factory.createClause(subj, verb, obj)

                coord = nlg_factory.createCoordinatedPhrase()
                coord.setConjunction(",")
                coord.addCoordinate(sent_1)
                coord.addCoordinate(sent)

                output = realiser.realiseSentence(coord)
        
            elif n == 0:
                sent_1 = nlg_factory.createClause("Absolutely yes my friend,")
                subj = nlg_factory.createNounPhrase("it")
                verb = nlg_factory.createVerbPhrase("be")
                obj = nlg_factory.createNounPhrase(f"the {random.choice(POSITIVE)} answer", f"of this {random.choice(FEATURES)}")
                sent = nlg_factory.createClause(subj, verb, obj)

                coord = nlg_factory.createCoordinatedPhrase()
                coord.setConjunction(",")
                coord.addCoordinate(sent_1)
                coord.addCoordinate(sent)

                output = realiser.realiseSentence(coord)

            return output
    """    

""""
    #Possibile lista delle Risposte negative che si possono dare quando una risposta non e' corretta
    def generate_negative_answer(self):
        n = random.randint(0, 2)
        if n == 2:
            sent_1 = nlg_factory.createClause("Absolutely no,")
            subj = nlg_factory.createNounPhrase("it")
            verb = nlg_factory.createVerbPhrase("be")
            obj = nlg_factory.createNounPhrase(f"the {random.choice(NEGATIVE)} answer. I am very disappointed in you")

            sent = nlg_factory.createClause(subj, verb, obj)
        
        
            sent_1 = nlg_factory.createClause("No,")
            subj = nlg_factory.createNounPhrase("You")
            verb = nlg_factory.createVerbPhrase("want")
            verb.addPostModifier(f"to be a padawan, because it is the {random.choice(NEGATIVE)} answer")

            sent = nlg_factory.createClause(subj, verb)
            sent.setNegated(True)
        
            sent_1 = nlg_factory.createClause("No,")
            subj = nlg_factory.createNounPhrase("it")
            verb = nlg_factory.createVerbPhrase("be")
            obj = nlg_factory.createNounPhrase(f"the {random.choice(NEGATIVE)} answer")
            sent = nlg_factory.createClause(subj, verb, obj)


            output = realiser.realiseSentence(sent)

        return output
"""

'"\n    #Possibile lista delle Risposte negative che si possono dare quando una risposta non e\' corretta\n    def generate_negative_answer(self):\n        n = random.randint(0, 2)\n        if n == 2:\n            sent_1 = nlg_factory.createClause("Absolutely no,")\n            subj = nlg_factory.createNounPhrase("it")\n            verb = nlg_factory.createVerbPhrase("be")\n            obj = nlg_factory.createNounPhrase(f"the {random.choice(NEGATIVE)} answer. I am very disappointed in you")\n\n            sent = nlg_factory.createClause(subj, verb, obj)\n        \n        \n            sent_1 = nlg_factory.createClause("No,")\n            subj = nlg_factory.createNounPhrase("You")\n            verb = nlg_factory.createVerbPhrase("want")\n            verb.addPostModifier(f"to be a padawan, because it is the {random.choice(NEGATIVE)} answer")\n\n            sent = nlg_factory.createClause(subj, verb)\n            sent.setNegated(True)\n        \n            sent_1 = nlg_factory.createCla

In [25]:
#ok Per ora
if __name__ == "__main__":
    question_test = GenerateText('questions')
    print(question_test.introduzione()) 
    print(question_test.start_questions())
    print("\n")

    for i in range(3):
        print(question_test.generate_text())
    print("\n")

    #answer_test = GenerateText('positive answers')

    for i in range(3):
        #print(answer_test.generate_answers())
        print(question_test.generate_good_answers())
    
    print('\n')

    #negative_test = GenerateText('negative answers')
    
    for i in range(3):
        #print(negative_test.generate_answers())
        print(question_test.generate_negative_answers())

    print("\n")

Hi stupid youngling, I am Darth Fener.
Well Youngling... and we talk about Padawian.


Is the Padawan the lowest order ?
Is the Jedi Knight always use violence ?
Is the Padawan the lowest order ?


Ok, it is the good answer of this attributes.
Maybe.., it is the true answer of this qualities.
Mh, it is the correct answer of this attributes.


You foul, it is the incorrect answer of this qualities.
Stupid youngling, it is the wrong answer of this attributes.
Stupid youngling, it is the wrong answer of this attributes.


